In [ ]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install motleycrew

Imports

In [1]:
import sys
import os

# Add the parent directory of the current file's location to the sys.path
sys.path.append(os.path.abspath("../src"))

from dotenv import load_dotenv
import neo4j
from ast import literal_eval
from queries import get_abstracts, get_abstract_pmids, get_functional_term_proteins
from summarization.article_graph import generate_embedding
from queries import cosine_similiarity
from crewai_tools import tool
from crewai import Crew, Process, Agent, Task
from crewai import Agent, LLM
from motleycrew import MotleyCrew
from motleycrew.agents.crewai import CrewAIMotleyAgent

Necessary functions

In [2]:
def get_driver():
    load_dotenv()

    # set config
    NEO4J_HOST = os.getenv("NEO4J_HOST")
    NEO4J_PORT = os.getenv("NEO4J_PORT")
    NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
    NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
    # connect
    uri = f"bolt://{NEO4J_HOST}:{NEO4J_PORT}"
    driver = neo4j.GraphDatabase.driver(uri, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    return driver

def abstracts_for_agent(query):
    query = literal_eval(query)
    driver = get_driver()
    test = get_abstracts(driver=driver, species="Mus_Musculus", query=query)
    abstracts =[{i["PMID"]: i["abstract"]} for i in test]
    driver.close()
    return abstracts

@tool("abstracts fetcher")
def fetch_abstracts(query: list, question: str):
    """"Fetches abstracts relevant for question about proteins, not functional terms. First argument is the query in format of a list, second is the question as a string."""
    driver = get_driver()
    pmids = get_abstract_pmids(driver=driver, species="Mus_Musculus", query=query)
    pmids =[i["PMID"] for i in pmids]
    embedded_query = generate_embedding(query=question)
    abstracts = cosine_similiarity(driver=driver, pmids=pmids, embedding=embedded_query)
    if len(abstracts) == 0:
        return "No abstracts found, maybe use another tool?"
    return " \n ".join(abstracts)

@tool("protein_from_functional_term_fetcher")
def fetch_proteins_from_functional_terms(funct_term: list) -> list:
    "Queries neo4j to retrieve proteins associated to functional terms."
    driver = get_driver()
    proteins = get_functional_term_proteins(driver, funct_term)
    driver.close()
    return proteins

Agents

In [3]:
llm = LLM(model="ollama/llama3.1", base_url="http://localhost:11434", temperature=0)
class CustomAgents:
    def __init__(self):
        self.Ollama = llm

    def helper_agent(self):
        return Agent(
            role="Helper",
            goal="Help the team",
            backstory="You're an expert biologist, you're here to help the team",
            llm=self.Ollama,
            max_iter=1
        )
    
    def protein_from_functional_term_fetcher_agent(self):
        return Agent(
            role="Functional term to protein fetcher",
            goal="You query the database to retrieve proteins associated to functional terms, functional terms have the format funct_term~source",
            backstory="You're an expert biologist, you focus on retrieving proteins associated to functional terms",
            allow_delegation=True,
            llm=self.Ollama,
            tools=[fetch_proteins_from_functional_terms],
            max_iter=1
        )
    def abstract_fetcher_agent(self):
        return Agent(
            role="Information fetcher",
            goal="Fetch abstracts needed to answer questions regarding protein interactions or functions.",
            backstory="You're an expert biologist, you focus on retrieving scientific abstracts. You know that the ferch abstracts tool can be used to retrieve abstracts for a given question and has the format fetch_abstracts(query: list, question:str)",
            allow_delegation=True,
            llm=self.Ollama,
            tools=[fetch_abstracts],
            max_iter=1
        )

    def summarization_agent(self):
        return Agent(
            role="Information summarizer",
            goal="You summarize information",
            backstory="You're an expert biologist, your main field is protein/gene interactions",
            allow_delegation=True,
            llm=self.Ollama,
            max_iter=1
        )
    def manager_agent(self):
        return Agent(
            role="Manager",
            goal="You manage the team",
            backstory="You're an expert biologist, you manage a team of biologists. You know that functional terms have the format funct_term~source. The abstract_fetcher_agent retireves abstracts and the summarization agent summarizes them.",
            llm=self.Ollama,
            max_iter=1
        )

Tasks

In [4]:
# This is an example of how to define custom tasks.
# You can define as many tasks as you want.
# You can also define custom agents in agents.py
class CustomTasks:
    def bionet_helper_task(self, context):
        return Task(
            description = f"Help the user. The user input is: {context}",
            expected_output="An answer to the question",
            verbose=True
        )

Define agents, tasks and start the work

In [ ]:
# Define agents
agents = CustomAgents()

manager = agents.manager_agent()
abstract_fetcher_agent = agents.abstract_fetcher_agent()
summarization_agent = agents.summarization_agent()
helper_agent = agents.helper_agent()
protein_fetcher_agent = agents.protein_from_functional_term_fetcher_agent()

# Define tasks
tasks = CustomTasks()

task = tasks.bionet_helper_task("whats the role of cd40?")
#task = tasks.bionet_helper_task("What are proteins associated in PWY-5910~BIOCYC and PWY0-1305~BIOCYC")

# Define the crew
crew = Crew(agents=[protein_fetcher_agent, summarization_agent, abstract_fetcher_agent], tasks=[task], process=Process.hierarchical, manager_agent=manager, verbose=True)

# Start the crew's work
result = crew.kickoff()
print(result)


# Agent: Manager
## Task: Help the user. The user input is: whats the role of cd40?
# Agent: Information fetcher
## Task: fetch information about cd40


# Agent: Information fetcher
## Thought: Thought: I need to fetch abstracts relevant for questions about proteins, specifically cd40.
## Using tool: abstracts fetcher
## Tool Input: 
"{\"query\": [\"cd40\"], \"question\": \"What are the functions of cd40 protein?\"}"
## Tool Output: 
The CD40 receptor is expressed constitutively on B lymphocytes, for which it provides important signals regulating clonal expansion, antibody production and isotype switching, as well as the development of humoral memory. The major source of CD154, the ligand for CD40, is activated T lymphocytes. Interactions between CD40 and CD154 provide a number of signals that play important roles in regulating the complex and multifactorial interactions between these two major cell types of the adaptive immune response. Understanding both the biological effects of thi

In [2]:
from llama_index.core.agent import ReActAgent
from motleycrew.tools import MotleyTool, RetryConfig
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.common.llms import init_llm
from motleycrew.common import LLMFramework, LLMProvider

In [4]:
llm = init_llm(
    llm_framework=LLMFramework.LANGCHAIN,
    llm_provider=LLMProvider.OLLAMA,
    llm_name="llama3.1",
    llm_temperature=0
)
@tool("abstracts fetcher")
def fetch_abstracts(query: list, question: str):
    """"Fetches abstracts relevant for question about proteins, not functional terms. First argument is the query in format of a list, second is the question as a string."""
    driver = get_driver()
    pmids = get_abstract_pmids(driver=driver, species="Mus_Musculus", query=query)
    pmids =[i["PMID"] for i in pmids]
    embedded_query = generate_embedding(query=question)
    abstracts = cosine_similiarity(driver=driver, pmids=pmids, embedding=embedded_query)
    if len(abstracts) == 0:
        return "No abstracts found, maybe use another tool?"
    return " \n ".join(abstracts)
fetch_abstracts_tool = MotleyTool.from_supported_tool(fetch_abstracts)
#fetch_proteins_from_functional_terms_tool = MotleyTool.from_supported_tool(fetch_proteins_from_functional_terms)

In [5]:
abtsract_fetcher = CrewAIMotleyAgent(
    llm=llm,
    role="Abstract Fetcher",
    goal="Fetch abstracts needed to answer questions regarding protein interactions or functions.",
    backstory="""You are an expert biologist.
    Your expertise lies in fetching abstracts.
    """,
    verbose=True,
    tools=[fetch_abstracts_tool],
)

# You can give agents as tools to other agents
react_agent = ReActToolCallingMotleyAgent(
    llm=llm,
    name="User helper",
    prompt_prefix="""You answer the user's question.""",
    tools=[abtsract_fetcher],
    verbose=True,
)

In [6]:
# You always need a crew to orchestrate the agents
crew = MotleyCrew()

blog_post_task = SimpleTask(
    crew=crew,
    name="Answer the question",
    description="""What's the role of cd40?""",
    agent=react_agent,
)
result = crew.run()
final_result = blog_post_task.output

2024-11-27 11:36:32,893 - motleycrew - WARNING - Lunary public key is not set, tracking will be disabled




> Entering new AgentExecutor chain...


> Finished chain.


In [24]:
final_result

''